## General Rules:
- This is an open book examination.
- Students may make use of a calculator.
- This is an online examination where you will access a computer; however you may not communicate with other students in any form.
- Headphone are prohibited.
- The use of AI (chatGPT etc.) is prohibited.
- All cell phones are to be switched off for the duration of the exam.
- The invigilator will not assist you with the explanation of questions.
- Students are prohibited from conversing in any manner with other students.

## My Name and Surname

Name =
</br>
Surname =  

### SECTION A: Theory Predicts Multiple choice

#### Question 1/10

Brooklyn’s housing authority has launched a crackdown on illegal short-term 
rentals. CityBnB’s internal data reveals clusters of listings in Williamsburg and 
Bushwick suspected to be operated by commercial entities posing as individual 
hosts. Your task is to identify hosts with multiple listings to help investigators 
prioritize enforcement. 

##### Which SQL clause would you use? 

>GROUP BY host_id HAVING COUNT(*) > 1 

#### Question 2/10 

CityBnB’s leadership is preparing a presentation for New York City Council to 
demonstrate compliance with short-term rental laws. They need a month-by-month 
breakdown of 2024 bookings to prove seasonal demand aligns with housing 
availability regulations. The bookings table uses YYYY-MM-DD formatting. 

##### Which query generates the required report? 

> SELECT MONTH(booking_date), COUNT(*) 
> FROM bookings 
> WHERE YEAR(booking_date) = 2024 
> GROUP BY MONTH(booking_date); 

#### Question 3/10

Chinatown’s community board reports a surge in vacant "ghost listings" that sit 
empty year-round, exacerbating housing shortages. CityBnB must calculate the 
percentage of listings marked "available" in the calendar table but with zero 
bookings to address public concerns. 

##### Which method calculates this accurately?

> (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM calendar) 

#### Question 4/10 

CityBnB is partnering with a real estate analytics firm to study housing trends. The 
firm requests a non-duplicated list of property types (e.g., "entire apartment," 
"private room") to analyze market saturation in high-demand areas like Manhattan 
and Queens.

##### Which SQL clause ensures unique property types?

>DISTINCT

#### Question 5/10 

A whistleblower in Sandton Heights—a luxury rental hotspot—claims that 30% of 
high-end listings are fake, using stock photos and fabricated reviews. CityBnB’s 
fraud team needs to identify the top 5 hosts by listing count for further 
investigation. 

##### What does this query detect? 

> Hosts with multiple listings, possible commercial operators

#### Question 6/10 

Users complain that CityBnB’s search filters fail to show affordable options in 
Harlem. The product team adds a "Budget-Friendly" filter for listings under 
$300/night but needs to validate the query before deployment. 

##### Which SQL query works?

>SELECT * FROM listings WHERE price < 300;

#### Question 7/10 

Following a data breach, CityBnB’s security team mandates an audit of hosts with 
unverified identities. The host_identity_verified column uses 'True'/'False' strings. 
Failure to comply could result in fines under New York’s short-term rental laws. 

##### Which SQL expression counts unverified hosts? 

>COUNT(CASE WHEN host_identity_verified = 'False' THEN 1 END)

#### Question 8/10 

A viral TikTok video exposes a "party house" in Astoria with 25+ noise complaints. 
CityBnB’s legal team needs to flag listings with >10 "noise" or "party" reviews AND >90% occupancy 
(booked ≥329 days/year) to avoid liability. 

##### Which query meets both conditions?

>WITH problematic_listings AS ( 
SELECT listing_id, COUNT(*) AS noise_reports 
FROM reviews 
WHERE comment ILIKE '%noise%' OR comment ILIKE '%party%' 
GROUP BY listing_id 
HAVING COUNT(*) > 10 
) 
SELECT l.id, l.name, p.noise_reports 
FROM listings l 
JOIN problematic_listings p ON l.id = p.listing_id 
WHERE l.availability_365 < 36;  -- 365 - 36 = 329 days booked

#### Question 9/10 

Scenario: A junior analyst’s query crashes CityBnB’s dashboard during a live demo 
to investors. The intended goal was to display high-rated properties (>4 stars) in 
Tribeca for a premium marketing campaign. 

##### What’s wrong with this syntax? 
- SELECT * FROM properties WHERE rating > 4 ORDER;

>The keyword ORDER should be ORDER BY

#### Question 10/10 

Scenario: As CityBnB scales to 50+ cities, redundant "host_status" entries (e.g., 
10,000 rows with 'superhost') slow down updates. The engineering team proposes 
normalizing this into a separate host_status table. 

##### What’s the primary advantage?

>To avoid redundant data and simplify updates 

### SECTION B: Practical questions with applied multiple choice
You are provided with a pre-populated SQLite database named `airbnb.db`. Download [here](https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata) if you haven't already. Your task is to explore this database and write a series of SQL queries to perform the tasks detailed below. Queries should be optimised to run within 20 seconds or less.

The tables and columns included in the `airbnb.db` are:

- `listings`: `id`, `host_id`, `name`, `neighbourhood_id`, `latitude`. `longitude`, `room_type_id`, ` construction_year`, `number_of_reviews`, `last_review`, `reviews_per_month`, `review_rate_number`, `calculated_host_listings_count` ,  `availability_365`, `instant_bookable`, `cancellation policy`, `house_rules`, `license`  
- `hosts`: `id`, `name`, `identity_verified`
- `neighbourhoods`: `id`, `name`, `neighbourhood_group_id`
- `neighbourhood_groups`: `id`, `name`
- `room types`: `id`, `type`
- `cancellation_policies`: `id`, `policy`

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///airbnb_nyc.db

### Q1/20: Find the total neighbourhoods in the airbnb listings

Map out the geographic scope of Airbnb presence.


In [5]:
%%sql

SELECT COUNT(DISTINCT id) AS total_neighbourhood
FROM neighbourhoods;

 * sqlite:///airbnb_nyc.db
Done.


total_neighbourhood
218


##### Q2/20: Which neighborhoods are experiencing the highest concentration of listings?

Find how listings are distributed across neighborhoods.

In [9]:
%%sql
SELECT 
    n.name AS neighbourhood,
    COUNT(*) AS total_listings
FROM listings l
JOIN neighbourhoods n ON l.neighbourhood_id = n.id
GROUP BY n.name
ORDER BY total_listings DESC
LIMIT 10;

 * sqlite:///airbnb_nyc.db
Done.


neighbourhood,total_listings
Bedford-Stuyvesant,3244
Williamsburg,3158
Harlem,2269
Bushwick,1984
Hell's Kitchen,1525
East Village,1489
Upper West Side,1486
Upper East Side,1454
Crown Heights,1287
Midtown,1057


In [13]:
%%sql

SELECT n.name, COUNT(l.id) AS total_listings
FROM listings l
JOIN neighbourhoods n ON l.neighbourhood_id = n.id
GROUP BY n.name
HAVING COUNT(l.id) > 1000
ORDER BY total_listings DESC;

 * sqlite:///airbnb_nyc.db
Done.


name,total_listings
Bedford-Stuyvesant,3244
Williamsburg,3158
Harlem,2269
Bushwick,1984
Hell's Kitchen,1525
East Village,1489
Upper West Side,1486
Upper East Side,1454
Crown Heights,1287
Midtown,1057


#### Q3/20: 

**1. Listing Availability**

**1.1 How many listings are there in the listings table?**
This query calculates the total number of `listings` available in the `listings table`. It provides a summary `count`, which is helpful for understanding the dataset’s size.

In [ ]:
%%sql

SELECT COUNT(*) AS total_listings
FROM listings;


 * sqlite:///airbnb_nyc.db
Done.


total_listings
39415


**2. Pricing Patterns**

**2.1 What is the average price of all listings?**
This query computes the average `price`, giving a sense of the typical cost of an Airbnb in San Francisco.


In [6]:
query = """
        SELECT AVG(price) AS average_price
        FROM listings;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,average_price
0,623.597564


**2.2 What is the most expensive and cheapest listing?**
This query finds the `listing` with the `highest price` and the `lowest price`.

[ ]


In [7]:
# Most expensive listing
query = """
        SELECT id, name, price
        FROM listings
        ORDER BY price DESC
        LIMIT 1;"""
p = pd.read_sql(query, conn)

# Least expensive listing
query = """
        SELECT id, name, price
        FROM listings
        ORDER BY price ASC
        LIMIT 1;"""
q = pd.read_sql(query, conn)

# Show both results
print("Most Expensive Listing:")
print(p)

print("\nLeast Expensive Listing:")
print(q)

Most Expensive Listing:
        id                               name   price
0  2431241  Beautiful Central Harlem sleeps 4  1200.0

Least Expensive Listing:
        id                               name  price
0  1200164  MANHATTAN Neat, Nice, Bright ROOM   50.0


**2.3 Which room type has the highest average price?**
This query helps identify which `room_type` tends to be `priced higher`.

In [8]:
query = """
        SELECT type, AVG(price) AS average_price
        FROM listings
        JOIN room_types ON listings.room_type_id = room_types.id
        GROUP BY type
        ORDER BY average_price DESC
        LIMIT 1;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,type,average_price
0,Shared room,650.88


**3. Host Activity**

**3.1 How many hosts are in the dataset?**
This query counts all `distinct hosts` in the dataset, helping measure host participation.

In [9]:
query = """
        SELECT COUNT(DISTINCT host_id) AS total_hosts
        FROM listings;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,total_hosts
0,39414


**3.2 How many superhosts(Exceptional hosts considered to have at least 4 reviews per month and number of reviews greater than 50) are there?**
This query counts how many hosts are marked as superhosts.

In [10]:
query = """
        SELECT DISTINCT host_id, name
        FROM listings
        WHERE reviews_per_month >= 4 AND number_of_reviews > 50;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,host_id,name
0,-763918612,PRIVATE Room on Historic Sugar Hill
1,-1694027102,☆Massive DUPLEX☆ 2BR & 2BTH East Village 9+ Gu...
2,1208474933,Astoria-Private Home NYC-
3,1363927884,Hospitality on Propsect Pk-12 yrs Hosting Lega...
4,387201125,yahmanscrashpads


**3.3 Which host has the most listings?**
This query identifies the host with the greatest number of listings on the platform.

In [11]:
query = """
        SELECT l.host_id, h.name , COUNT(*) AS listing_count
        FROM listings l
        JOIN hosts h ON l.host_id = h.id
        GROUP BY host_id
        ORDER BY listing_count DESC
        LIMIT 1;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,host_id,name,listing_count
0,-1148768701,Yolanda,2


**3.4 Which hosts have listings in more than one neighbourhood?**
This identifies multi-location hosts, possibly professional operators.

In [12]:
query = """
        SELECT host_id
        FROM listings
        GROUP BY host_id
        HAVING COUNT(DISTINCT neighbourhood_id) > 1;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,host_id
0,-1148768701


**4. Neighbourhood Performance**

**4.1 Which neighbourhoods have listings?**
This query shows all `neighborhoods` with `at least one listing`.

In [13]:
query = """
        SELECT DISTINCT name
        FROM neighbourhoods;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,name
0,Allerton
1,Arden Heights
2,Arrochar
3,Arverne
4,Astoria


**4.2 Which neighbourhoods have the highest number of listings?**
This query counts how many listings exist in each neighbourhood. It helps identify the most active or popular neighbourhoods for Airbnb activity.

In [14]:
query = """
        SELECT l.neighbourhood_id, COUNT(*) AS total_listings
        FROM listings l
        GROUP BY l.neighbourhood_id
        ORDER BY total_listings DESC
        LIMIT 5;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,neighbourhood_id,total_listings
0,5,3244
1,10,3158
2,11,2269
3,8,1984
4,6,1525


**4.3 What are the different room types available?**
This query lists all `unique room_types` available in the dataset.

In [15]:
query = """
        SELECT DISTINCT type
        FROM room_types;"""
q = pd.read_sql(query, conn)

# Show result
q.head()

,type
0,Entire home/apt
1,Private room
2,Shared room


**5. Review Behaviour**

**5.1 Which listings are available every day of the year and have more than 50 reviews?**
This query filters for high-availability, high-activity listings.


In [16]:
query = """
        SELECT id, name
        FROM listings
        WHERE availability_365 = 365 AND number_of_reviews > 50;"""
q = pd.read_sql(query, conn)

# Show result
q.head()


,id,name
0,2489785,"1Bedroom, Seconds from L train"
1,4111337,"Bklyn, private Three Bedroom."
2,4929294,Convenience & Chill
3,6683951,Putnam Garden -2BDR Garden Apt
4,6701624,Clean Cozy & Comfy Apartment!
